In [1]:
import matplotlib.pyplot as plt
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import GOOG, SMA


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

/home/tarasnovokhatskiy/long-live-bitcoin/.venv/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [135]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from tqdm import trange
from unicorn_binance_rest_api import BinanceRestApiManager as RestManager


def load_history(client: RestManager, *, pair_sym='BTCUSDT', t_start: int, t_end: int, interval: str) -> pd.DataFrame:
    def to_numbers(l: list):
        return [int(i) if isinstance(i, int) else float(i) for i in l]

    def to_datetime(ts):
        return datetime.fromtimestamp(ts / 1e3).date()
        # return datetime.strptime(s[:-3], '%Y-%m-%d %H:%M:%S.%f')

    # TODO: make flexible progress step
    timeframe_delta = int(timedelta(days=60).total_seconds())
    result = []
    t_start = int(t_start)
    t_end = int(t_end)
    progress = trange(t_start, t_end, timeframe_delta)
    for ts in progress:
        from_ts = datetime.fromtimestamp(ts)
        to_ts = min(datetime.fromtimestamp(t_end), datetime.fromtimestamp(ts + timeframe_delta))
        progress.set_postfix(from_date=str(from_ts), to_date=str(to_ts))
        candles = client.get_historical_klines(pair_sym, interval, str(from_ts), str(to_ts))
        result += candles

    result = [to_numbers(i) for i in result]
    result = np.array(result)
    result = pd.DataFrame({
        'Date': result[:, 6],
        # 'OpenTime': result[:, 0],
        'Open': result[:, 1],
        'High': result[:, 2],
        'Low': result[:, 3],
        'Close': result[:, 4],
        'Volume': result[:, 5],
        # 'CloseTime': result[:, 6],
        'QuoteAssetVolume': result[:, 7],
        'NumOfTrades': result[:, 8],
        'TakerBuyAssetVolume': result[:, 9],
        'TakerSellAssetVolume': result[:, 10],
    })

    result.Date = result.Date.apply(to_datetime)
    result.Date = pd.to_datetime(result.Date)
    result.set_index('Date', inplace=True)
    return result

In [139]:
key = 'w9WBoIU2EYlGAUT0ddfEvaIc6E0r4KtuP54UKbGpTTdb1Bcg2stFnDBU8Z3e0aWw'
secret = 'g9vXgPlbEW2NGnsXuNGD6gkVr0BJEwGmhziDbZydQIV4fHcsP46lTrYs3KW7GjQf'

client = RestManager(key, secret)
today = datetime.now()
duration = timedelta(days=365*3)
btcusdt = load_history(
    client=client,
    pair_sym='BTCUSDT',
    t_start=(today - duration).timestamp(),
    t_end=today.timestamp(),
    interval='1d')

# btcusdt.Date = pd.to_datetime(btcusdt.Date)
# btcusdt = btcusdt.groupby(['Date']).agg({
#     'Open': 'first',
#     'Close': 'last',
#     'Volume': 'sum',
#     'Low': 'min',
#     'High': 'max'
# })
btcusdt.head()

  0%|          | 0/19 [00:00<?, ?it/s, from_date=2019-04-11 14:41:39, to_date=2019-06-10 14:41:39]/home/tarasnovokhatskiy/long-live-bitcoin/.venv/lib/python3.8/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
100%|██████████| 19/19 [00:19<00:00,  1.01s/it, from_date=2022-03-26 13:41:39, to_date=2022-04-10 14:41:39]


,Open,High,Low,Close,Volume,QuoteAssetVolume,NumOfTrades,TakerBuyAssetVolume,TakerSellAssetVolume
Date,,,,,,,,,
2019-04-13,5017.37,5080.58,4861.22,5048.01,33276.678614,1.664858e+08,226036.0,16899.590533,8.456182e+07
2019-04-14,5047.00,5099.00,5004.00,5045.22,17292.456802,8.737598e+07,138674.0,8692.053259,4.392529e+07
2019-04-15,5047.45,5152.99,5000.00,5131.30,18281.607739,9.259493e+07,131700.0,10043.911667,5.089395e+07
2019-04-16,5131.28,5167.38,4950.00,5024.95,29057.191581,1.476876e+08,204944.0,14170.252402,7.207017e+07
2019-04-17,5024.95,5197.72,5003.94,5173.72,24242.229493,1.235817e+08,168550.0,12804.633768,6.529062e+07


In [140]:
# btcusdt.set_index('Date', inplace=True)
# btcusdt

In [146]:
bt = Backtest(btcusdt, SmaCross, cash=10_000, commission=.002)
bt.run()
bt.plot();

/tmp/ipykernel_251275/1753268410.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(btcusdt, SmaCross, cash=10_000, commission=.002)


In [134]:
type(btcusdt.index)

pandas.core.indexes.base.Index